In [ ]:
#| default_exp power


In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd

# Direct Power Analysis

The calculating of the power transmitted to the water correcting for environmental conditions is 

$$P_{Did} = P_{Dms}-\Delta P,$$ where $P_{Dms}$ is the delivered power in the trial conditions, and $\Delta P$ is the correction of the delivered power taking into account the resistances and changes in propulsive power as a result of the trial conditions.

This section of the `pyseatrials` library is concerned with deriving the values needed for this calculation


Some things should come with a bit more clarification e.g. 'wake-fraction', what is it?

## Understanding the notation

There are severl sub-scripts used in this section which are useful to understand

There are four types of upper case letter used which require further explanation


- **D** 'Delivered' used to differentiate types of power 
- **M**  'Model' used for values taken from a model of the ship in a tank test
- **R** 'Relative' used to differentiate different types of efficiency 
- **S** 'Full-scale' used to distinguish from values derived from models

The lower case abreviations are

- **id** refers to the 'ideal condition'
- **ms** refers to the 'trial condition'


In [ ]:
#| export
import numpy as np
from fastcore.test import *

## Correction delivered power

Calculates the power when correcting for the resistances experienced by the vessel as well as the deciation of propeller efficiency relative to ideal

$$\Delta P = \frac{\Delta R V_s}{\eta_{Dms}}+ P_{Dms}(1-\frac{\eta_{Dms}}{\eta_{Did}}), $$ where $\Delta R$ is the total resitance increase experienced by the ship, $V_s$ is the speed through water, $\eta_{Did}$ is the propulsive efficiency coefficient in the ideal conditions, $P_{Dms}$ is the shaft power during the test,$\eta_{Dms}$ is the propulsive efficiency coefficient in the trial conditions.

The value for $\Delta R$ is calculated as the some of the wind, wave, and water resistances. The functions for these values are found at `wind_resistance`, `stawave1_fn`, and `temp_salinity_water_resistance`

**ITTC equations**: J-2

In [ ]:
#| export
def correction_delivered_power(
    p_dms:float, #delivered power [W]
    resistance_increase:float, #Resistance increase derived from data measured in seatrial
    stw:float, #speed through water [m/s]
    eta_id:float, #propulsive efficiency in the ideal conditions
    eta_ms:float, # propulsive efficiency in the seattrial

)-> float:# Returns the corrected delivered power in Newtons [N]
    
    "calculates the corrected delivered power, used as part of the direct power analysis"
    
    return resistance_increase * stw /eta_id + p_dms * (1- eta_ms/eta_id)
    

Example of using function, generally this is not used directly

In [ ]:
correction_delivered_power(1e4, 1e3, 10, 0.8, 0.7)

13750.000000000002

## Propulsive efficiency correction
This can be used to calculate both the ideal and trial conditions reffered to in equations J-3 and J-5
$$\eta_D = \eta_o \eta_R \frac{1-t}{1-w_s} $$

**ITTC equations** J-3, J-5


In [ ]:
#| export
def propulsive_efficiency_corr(n_o:float, #open water efficiency
                          n_r:float, #relative rotative efficiency
                          t:float, #thrust deduction factor
                         w_s:float, #full-scale wake fraction
                         ):
    "Calculates propeller efficiency adjusting for additional resistances"
    return n_o*n_r*(1-t)/(1-w_s)
    

In [ ]:
propulsive_efficiency_corr(0.58, 0.7, 0.1, 0.5)

0.7308

In [ ]:
#| hide

test_eq(propulsive_efficiency_corr(0.58, 0.7, 1, 0),0)
test_eq(propulsive_efficiency_corr(1, 1, 0, 0.5),2)
#checks to make sure there is a divide by 0 error
test_fail(lambda: propulsive_efficiency_corr(0.58, 0.7, 0.0, 1),  contains="division by zero" )


## Full scale wake fraction

There are two different functions related to the full scale wake fraction, both are presented in this sub-section. 

The two related but distinct values are

- $w_{S}$: Full-scale wake fraction using model
- $w_{S}$: Full-scale wake fraction using speed
- $e_i$ Scale correlation factor of the wake faction

### The full scale wake fraction

$$w_S =  1- (1- w_M)e_i,$$ where $W_S$ is the fullscale wake fraction, $W_m$ is the model wake fraction derived from tank tests, and $e_i$ is the scale correlation factor derived from **XXXX derived from what?**.

**ITTC equation**: J-4, J-20


In [ ]:
#| export
def full_scale_wake_fraction(wake_fraction_model:float,
                            scale_correlation_factor:float
                            )-> float:
    
    "used to scale from model results to full-scale vessel"
    
    return 1- (1- wake_fraction_model) * scale_correlation_factor

The full scale wake fracrtion returns a value between 0 and 1

In [ ]:
full_scale_wake_fraction(0.4, 0.8)

0.52

In [ ]:
#| hide

test_eq(full_scale_wake_fraction(1, 0.8), 1)
test_close(full_scale_wake_fraction(0, 0.8), 0.2, eps = 1e-6)

### Full-scale wake fraction from speed

This approach calculates the wake fraction using the measured water speeds

$$w_S =1- \frac{V_A}{V_S},$$ where $V_A$ is the speed of flow into the propeller, and $V_S$ is the ship's speed through water. 

**ITTC equations**: J-17

In [ ]:
#| export

def full_scale_wake_speed(flow_speed:float, #The speed of flow through the propeller
                         stw:float, #Ship's speed through water
                         )-> float:
    
    "Calculate the wake fraction using the measured water speeds"
    
    return 1 - (flow_speed/stw)

This approach to obtianing the full scale wake speed is typically used for getting the trial conditions

In [ ]:
full_scale_wake_speed(10,50)

0.8

In [ ]:
#| hide
test_eq(full_scale_wake_speed(50,50),0)

### Scale correlation factor

The scale correlation factor is simply the re-arranged fullscale wake fraction. And is shown as

$$e_i = \frac{1-w_S}{1-w_M},$$ where values are the same as previously.

**ITTC equation**: J-21

In [ ]:
#| export
def scale_correlation_factor(
    trial:float, #The full-scale wake fraction in the trial
    model:float  #The wake fraction of the model derived from tank tests
)-> float: #The dimensionless coefficient joining the full scale and model fractions
    "Calcualte the scale correlation factor using the model fraction from tank tests, and the full-scale fraction from trials"
    return (1  - trial)/(1-model)

Deriving the scale correlation factor is easy given the inputs

In [ ]:
scale_correlation_factor(0.6,0.8)

2.0000000000000004

In [ ]:
#| hide

test_eq(scale_correlation_factor(1,0.8),0)

## Self propulsion factors

This function allows for ship factors to be adjust accounting for the difference between ideal/model conditions and the conditions during the trial. This function is equivalent to equation J-6 to J-8.
The function is essentially the following equation

$$x_{test} = x_{ideal} + \Delta x_R (\frac{\Delta R}{R_{ideal}}),$$
Where $x$ is the variable to be adjusted $x_{ideal}$ is the value of the variable under ideal conditions, $\Delta x_R$ the coefficient of of change for each unit of $\frac{\Delta R}{R_{ideal}}$, $\Delta R$ the resistance increase derived from data measured during the sea trial and $R_{ideal}$ is the resistance under ideal conditions. The values of $x_{ideal}$ are obtained from a model test. The value of $\Delta x_R$ is found by fitting a linear model using data gathered during a specific tank tests. The process of obtaining the value of $x_R$ is described in detail in section J.2.

In practice however, these sets of values are not needed as the deviations are often negligable in comparison to the variation of $\eta_O$. That using 
$$ x_{test} \approx x_{ideal},$$
is acceptable

**ITTC equations**: J-6, J-7, J-8

In [ ]:
#| export
def self_propulsion_factors(
    x_ideal:float, #The variable in ideal conditions. It is acceptable to use this value without adjustments
    delta_x:float = 0, #The change per unit of the resistance ratios. Default is 0
    delta_r:float = 1, #increase in resistance from ideal conditions
    delta_r_ideal:float = 1 #Resistance in ideal conditions
) -> float:
    
    "Adjusting the self propulsion factors is only possible if the required model tests have been performed. By default this function returns the ideal value"
    
    return x_ideal + delta_x * (delta_r/delta_r_ideal)

Generally the adjustment for the self propulsion factors will be small. But if available should be applied

In [ ]:
self_propulsion_factors(0.8, 0.1, 1000, 10000)

0.81

In [ ]:
#| hide

test_eq(self_propulsion_factors(0.8), 0.8)


## Calculate Thrust coefficient, Torque coefficient, and Load factor

The thrust coefficient, torque coefficient and the load factor provide useful values for calculating the adjusted propeller efficiency. There are several different approaches. All three approaches are described in this sub-section and are used at various points in the process.

The different approaches are

- The inverse quadratic method
- The quadractic method
- Torque coefficient from water conditions
- Load factor using propeller advance and thrust
- Load factor using resistance


### The inverse quadratic method

The inverse quadratic method doesn't actually calculate the coefficients themselves but using them as the dependent variable of a quadratic curve where the dependent variable is the propeller advance coefficient in the ideal condition. The result is that the coefficients of the quadratic curve can be either be solved to find the point closest to 0 where the line crosses the x-axis. Alternatively the quadratic coefficients are used replacing the propeller advance coefficients in the ideal condition with those of the trial condition, this creates a quadratic equation that return the thrust coefficient in the trial condition.
The propeller advance coefficients in the ideal condition are provided by tank tests, usually 10 data points are supplied in order to fit the curve

$$y = aX^2 + bX +c$$

The coefficients of this model ($a,b,c$) are not useful inthemselves but are input parameters to other functions for example `torque_coef`

The equation is also used to calculate the thrust coefficient in the trial condition/


**ITTC equations**: J-9, J-10, J-11, J-25, J-26



In [ ]:
#| export
def get_curve_coefficient(y:float, #An array containing the dependent variable coefficient
                      x:float, #An array containing the propeller advance coefficient
                     )->float: #Returns an array containing model coefficients
    
    "Obtain the coefficients used to calculate the Thrus, and Torque coefficients and the load factor coefficients"
    
    #create the X matrix to have a quadratic form
    X = np.concatenate((x**2,x, np.ones(len(x)))).reshape([3,len(x)]).transpose()
    #Get determinate

    square_mat = np.matmul(X.transpose(), X)

    detX = np.linalg.inv(square_mat)

    temp = np.matmul(detX, np.transpose(X))
    #Return the beta value
    b = np.matmul(temp, y)
    
    return b
    

For ship whose model tests give a propeller advance coefficient to thrust coefficent $K$ relationship as shown below, we can (back) calculate the coefficients of the quadratic formula using the get_curve_coefficient function.

In [ ]:
J = np.linspace(1,10, 10)
K = J**2 + 2*J +3 #In reality we obviously do not know the coefficients before hand!
get_curve_coefficient(K, J)

array([1., 2., 3.])

When calculating the loading factor coefficients $\tau$ the values for the propeller advance coefficient needs to be inverted

In [ ]:
J = 1/np.linspace(1,10, 10)
K = J**2 + 2*J +3 #In reality we obviously do not know the coefficients before hand!
get_curve_coefficient(K, J)

array([1., 2., 3.])

In [ ]:
#| hide
#tests are amazing, found out I had an error in the code because the test failed at seemingly random points.
#was actually an error in the function causing it to search for the variable in the global environment
x = np.linspace(1,10, 10)
y = x**2 + 2*x +3
test_close(get_curve_coefficient(y, x)[1], 2, eps = 1e-5)

#there was an error which occured when the array was shaped to hard coded values 3,10 when the vector was not 10 error was thrown
#I fixed this by adding len(x) not sure if or how to build a test round this


### The quadratic method

The quadratic method assumes you have obtained the coefficients using the inverse quadratic method described previously. This approach simply plugs the value of propeller advance ($J$) into the equation to return the value of the target coefficients/ factor. This is typically used to calculate the coefficents in the trial condition

In [ ]:
#| export
def quadratic_method(coefs:float, #An array of the coefficients created by the function get_curve_coefficient
                    propeller_advance_coef:float #The propeller advance coefficient
                    )-> float: #The target value for the coefficient types entered
    
    "Calculate the coefficient using a modelled quadratic curve"
    
    return coefs[0] * propeller_advance_coef**2 + coefs[1] * propeller_advance_coef + coefs[2]

The ITTC use the quadratic method to calculate the thrust coefficient $K_{Qms}$ in the trial condition.

In [ ]:
quadratic_method([1,2,3], 2)

11

In [ ]:
#| hide

test_eq(quadratic_method([1,2,3], 2), 11)

### Torque coefficient from water conditions

The torque coefficient is obtained using 

$$K_Q = \frac{P}{2 \pi \rho n^3 D^5} \eta,$$ Where $P$ is power, $n$ is rotations per second, $D$ is shaft diameter and $eta is relative rotative efficiency.

Generally the torque coefficient for the ideal condition is known through tank tests. However, this equation is also used to find the thrust coeficient during the trial. When the trial thrust coefficient is calculated $P$, $n$, and $\eta$ are all from trial data.

**ITTC equations**: J-12

In [ ]:
#| export
def torque_coef(power:float, #The delivered power
                     shaft_speed:float, #measure propeller shaft speed [rev/s]
                     diameter:float, #properller_diameter [m]
                     efficiency:float, #relative rotative efficiency
                     water_density:float = 1026 #water density [kg/m^3]
                     )->float: #dimensionless thrust coefficient
    
    "calcualte the torque coefficient under ideal or trial conditions"
    
    denominator = 2 * np.pi * water_density * shaft_speed**3 * diameter**5
    
    return (power/denominator) *  efficiency
    
    

under trial conditions the thrust coefficient is calculated directly

In [ ]:
torque_coef(1e4, 50, 1.5, 0.8)

1.3073637822066664e-06

In [ ]:
#| hide

test_eq(torque_coef(1e4*np.pi, 1, 1, 1, 1000), 5)

### Load factor using propeller advance and thrust

The load factor on the propeller can be calculate by

$$\tau_{P} = \frac{K_{T}}{J^2},$$ where $K_{T}$ is the thrust coefficient and $J$ is the propeller advance coefficient. This approach is usally for finding 
$\tau_{P}$ the trial condition

**ITTC equations**: J-18

In [ ]:
#| export
def load_factor(thrust_coefficient:float, #The thrust coefficient
               propeller_advance:float #The propeller advance coefficient
               )->float: # dimensionless load factor
    
    "Calculate the load factor using the thrust and propeller advance coefficients"
    
    return thrust_coefficient/propeller_advance**2

The load factor on the propeller for the given ship conditions is a key element to calculating the corrected power delivered

In [ ]:
load_factor(0.8, 5)

0.032

In [ ]:
#| hide
#another empty test
test_eq(load_factor(4, 2), 1)

### Load factor using resistance and propellor state

This approach to calculating  the loading factors are 
$$ \tau = \frac{R}{(1- t)(1-w_S)^2 \rho V^2_S D^2},$$ where $R$ is the total resistance, $t$ is the thrust deduction factor, $w_S4$ is the full-scale wake fraction, $\rho$ is the water density, $V_S$ is the ship speed through water, and $D$ is the diameter of the propeller. This equation is the same as `total_resistance` but re-arranged to find $\tau$.

**ITTC equations**: J-23

In [ ]:
#| export
def load_factor_resistance(
                    resistance:float, # The total resistance experienced by the vessel
                    thrust_deduction:float, #The thrust deduction factor
                    wake_fraction:float, #The full-scale wake fraction
                    stw:float, #Ships speed through water [m/s]
                    diameter:float, #The diameter of the ships propeller
                    water_density:float = 1026, #density of water in the given conditions [kg/m^3]
    )-> float: #this value can be in the ideal condition or trial depending on parameters used
    
    "Calculate the load factor of the propeller. Usually used to find the load factor in the ideal condition"
    
    return resistance /( (1 - thrust_deduction) * (1- wake_fraction)**2 * water_density * stw**2 * diameter **2 )

The load factor using resistance allow the load factor in the ideal condition to be calculated using only basic information

In [ ]:
load_factor_resistance(1e4, 0.7, 0.8, 10, 2.3, 1026)

1.5353794413921111

In [ ]:
#| hide

test_eq(load_factor_resistance(1e4, 0.5, 0.5, 10, 1, 1), 800)

## Trial propeller advance

The propeller advance coefficient for the trial conditions can be calculated by factorising the coefficients of the quadratic curve founs using `get_curve_coefficient`. The factorisation is the standard quadratic method using

$$J = \frac{-b - \sqrt{b^2 -4a(c - K_Q)}}{2a},$$  or $$J = \frac{-b - \sqrt{b^2 -4(a- \tau)c}}{2(a- \tau)},$$ where $a,b,$ and $c$ are the ouput coefficients from `get_curve_coefficient` and $K_Q$ is the torque coefficient calculated using `torque_coef`, and $\tau$ is the load factor. Note that the propeller advance is only the negative part of the quadratic equation as properller advance must always be positive. This equation is used to derive the propeller advance coefficient from empirical measurements.

**ITTC equations**: J-13, J-24


In [ ]:
#| export
def propeller_advance_coefficient(propeller_value:float, #The torque coefficient or loading factor as appropriate
                                  a:float, #coefficient 'a' from get_curve_coefficient
                                  b:float, #coefficient 'b' from get_curve_coefficient
                                  c:float, #coefficient 'c' from get_curve_coefficient
                                  mode:str, #Does function use torque or load mode?
                                  ) -> float: 
    
    "Calculate the propeller advance coefficient using a quadratic curve"
    
    assert (mode == "torque") | (mode == "load")
    
    if mode =="torque":
        
        c = c - propeller_value
        
    elif mode == "load":
    
        a = a - propeller_value
    
    square_root = np.sqrt(b**2 - 4*a*c)
    J = (-b - square_root )/(2*a)
    
    return J

The propeller advance can then be calcualted by combining outputs of the previous functions `get_curve_coefficient` and `torque_coef` as inputs to `propeller_advance_coefficient`.


In [ ]:
#These values are obviously not realistic
J = np.linspace(1,10, 10)
K = 2*J**2 + -5*J +3 #In reality we obviously do not know the coefficients before hand!
curve_coefs = get_curve_coefficient(K, J)
trial_torque_coef = torque_coef(1e4, 50, 1.5, 0.8)
propeller_advance_coefficient(trial_torque_coef, curve_coefs[0], curve_coefs[1], curve_coefs[2], mode ='torque'  )

0.9999986926396782

Remember when calulating the propeller advance using load factors the coefficients used are not those from the load quadratic method but from Thrust

In [ ]:
#when I have some example data show that thrust is used not torque

propeller_advance_coefficient(trial_torque_coef, curve_coefs[0], curve_coefs[1], curve_coefs[2], mode ='load'  )

0.9999986926430968

In [ ]:
propeller_advance_coefficient(0, 2, -5, 3, mode = "load" )

1.0

In [ ]:
#| hide
#(x - 1)(2x - 3) = 2x^2 -5x +3
#put in test when I have reasonable values to test
#check the equation is implemented properly
test_eq(propeller_advance_coefficient(0, 2, -5, 3, mode = "torque" ),1)

## Open water efficiency

The efficiency of the properller in open water can be calcualted using 

$$\eta_O = \frac{J}{2 \pi}\frac{K_T}{K_Q},$$ where J is the propeller advance coefficient, $K_T$ is the thrust coefficient, and $K_Q$ is the torque coefficient. It can be used for calulating both the trial conditions, but is typically used for trial conditions as the ideal conditions are known from tank tests.

**ITTC equations**: J-15, J-27

In [ ]:
#| export
def open_water_efficiency(propeller_advance_coef:float, #The propeller advance coefficient of the ship
                         thrust_coef:float, # thrust coefficient
                         torque_coef:float 
                         )-> float:
    
    "Calculate the open water propeller efficiency"
    
    return (propeller_advance_coef/(2*np.pi))*(thrust_coef/torque_coef)

The proppeller open water efficiency for a ship with a propeller advance coefficent of 5, thrust coefficent of 0.8 and torque coefficient of 0.9. Is shown below

In [ ]:
open_water_efficiency(5, 0.8, 0.9)

0.707355302630646

In [ ]:
#| hide
test_eq(open_water_efficiency(5, 2*np.pi, 1), 5 )

## Speed of flow into the propeller

The rate at which water flows past the propeller is given by

$$V_A = JnD$$, where $J$ is the propeller advance coefficient, $n$ is the rotations per second, and $D$ is the diameter of the propeller. When calculating the speed under trial conditions the propeller advance coefficieant can be found using `propeller_advance_coefficient`

**ITTC equations**: J-16

In [ ]:
#| export
def propeller_flow(
    propeller_advance_coef:float, #Propeller advance coefficient [n/a]
    rotations_sec:float, #propeller rotations per second [rev/sec]
    diameter:float, #Diamter of the propeller [m]
    )-> float: #The value that comes out is in m3/s WHAT ARE THE UNITS?
    
    "Calculate speed of water flow into the propeller"
    
    return propeller_advance_coef * rotations_sec * diameter

really the propeller advance coefficient function is just a simple wrapper around the product of three values

In [ ]:
propeller_flow(5, 30, 2)

300

In [ ]:
#| hide

test_eq(propeller_flow(5, 30, 2),300)

## Total Resistance

The total resistance experienced by the ship system is given by.

$$R = \tau(1- t)(1-w_S)^2 \rho V^2_S D^2,$$ where $\tau$ is the loading factor, $t$ is the thrust deduction factor, $w_S4$ is the full-scale wake fraction, $\rho$ is the water density, $V_S$ is the ship speed through water, and $D$ is the diameter of the propeller. This equation is typically used to calculate the resistance in the trial conditions and is the same as `load_factor_resistance` but rearranged to get $R$. The total resistance in the trial conditions is used to calculate the resistance in the ideal conditions, using the following formula.$$R_{id} = R_{ms} - \Delta R,$$ where $R_{ms}$ is the resistance in the trial conditions, $\Delta R$ is the resistance created by the wind, wave, and water. 

**ITTC equations**: J-19

In [ ]:
#| export
def total_resistance(
                    load_factor:float, # The load factor
                    thrust_deduction:float, #The thrust deduction factor
                    wake_fraction:float, #The full-scale wake fraction
                    stw:float, #Ships speed through water [m/s]
                    diameter:float, #The diameter of the ships propeller
                    water_density:float = 1026 #density of water in the given conditions [kg/m^3]
    )-> float: #this value can be in the ideal condition or trial depending on parameters used
    
    "Calculate the total resistance of the ship. Used to find the resistance in the ideal condition"
    
    return load_factor * (1 - thrust_deduction) * (1- wake_fraction)**2 * water_density * stw**2 * diameter **2

total resistance of the ship ultimately decides it's maximum speed and the fuel consumption it has at a given speed

In [ ]:
total_resistance(5, 0.7, 0.8, 10, 2.3, 1026)

32565.23999999998

In [ ]:
#| hide

test_eq(total_resistance(1, 0.5, 0.5, 10, 1, 10), 125)

## Corrected propeller shaft speed

The corrected propeller shaft speed is $$\eta_ = \frac{V_S(1-w_{S})}{JD},$$ where $V_s$ is the speed of the vessel, $w_S$ is the full-scale wake fraction, $J$ is the propeller advance coefficient in the ideal condition, and $D$ is the diameter of the propeller. This equation is is a re-arranged version of  `propeller_flow` with the substitution $V_A = V_S (1-w_S)$

**ITTC equations**: J-28

In [ ]:
# | export
def propeller_speed(
        propeller_advance_coef:float, #Propeller advance coefficient [n/a]
        stw:float, #The speed through water of the vessel [m/s]
        diameter:float, #Diamter of the propeller [m]
        wake_fraction:float, #The full scale wake fraction
        )-> float: #Propeller speed in rotations per second
    "Calculate the propeller speed in m/s"

    return stw*(1-wake_fraction)/(propeller_advance_coef * diameter)

Once calculated propeller speed can be inserted into the beggining of the calculation process to iterate the model to convergence and find the final value for $\Delta P$

In [ ]:
propeller_speed(2, 10, 2, 0.5)

1.25

In [ ]:
#| hide

test_eq(propeller_speed(2, 10, 2, 0.5),1.25)

## Power in the ideal conditions


After finding all the intermediary stages the final corrected power values can be calcualted. $$P_{Did} = P_{Dms} - \Delta P$$

The calulation of these values is broken into two stages

- Trial: The calculations needed to get all the values relating to and required for values 'in the trial condition'
- Ideal: The calculations needed to get all the values relating to and required for values 'in the ideal condition'

The trial values are calculated first then the results are used to calculate the ideal values. once both phases have been completed the delivered power in the ideal condition can be found.

### Calculate values in the trial condition



In [ ]:
#| export
def calculate_all_values_from_trial_phase(
    V_s:float,
    P_dms:float,
    eta_ms:float,
    delta_R:float,
    R_id:float,
    delta_eta :float,
    delta_t:float, 
    delta_w:float,
    shaft_speed:float,
    diameter:float,
    e_i:float,
    t_Rid:float,
    w_Mid:float,
    number_shafts:float,
    K_T,
    K_Q,
    J,
    water_density:float = 1026
    ): #returns a dictionaryt of the ouput variables from the trial condition calculations
    
    "Perform all the calculations to get the 'trials conditions' values"
    #These return the initia; value of eta/t/w, the function is used just to show, although it does nothing. This can be swapped out
    eta_Rms = eta_ms
    t_ms = t_Rid
    w_Mms = w_Mid

    K_Qms = torque_coef(P_dms, shaft_speed, diameter, eta_Rms, water_density)/number_shafts #two shafts


    ## These two functions are the only bit that cannot be vectorised 
    ##However these values are fixed from the test data so should be fixed for each ship, so vectorisation should be irrelevant
    K_T_coeffs = get_curve_coefficient(K_T, J)
    K_Q_coeffs = get_curve_coefficient(K_Q, J)

    #not vectorised
    J_ms = propeller_advance_coefficient(K_Qms, K_Q_coeffs[0], K_Q_coeffs[1], K_Q_coeffs[2], "torque"  )


    K_Tms = quadratic_method(K_T_coeffs, J_ms)

    tau_Pms = load_factor(K_Tms  , J_ms)


    V_A = propeller_flow(J_ms, shaft_speed, diameter)

    w_Sms = full_scale_wake_speed(V_A, V_s)

    R_ms = total_resistance(tau_Pms, t_ms, w_Sms, V_s, diameter, water_density) *number_shafts #number of propellers

    R_id = R_ms - delta_R

    eta_Oms = open_water_efficiency(J_ms, K_Tms, K_Qms)  




    eta_Rms = self_propulsion_factors(eta_ms, delta_eta, delta_R, R_id)
    t_ms = self_propulsion_factors(t_Rid, delta_t, delta_R, R_id)
    w_Mms = self_propulsion_factors(w_Mid, delta_w, delta_R, R_id)

    eta_Dms = propulsive_efficiency_corr(eta_Oms, eta_Rms, t_ms, w_Sms)

    e_ims = scale_correlation_factor(w_Sms, w_Mms)

    w_Sid = full_scale_wake_fraction(w_Mid,e_ims)  
    
    trial_values = {'K_Qms':K_Qms, 'J_ms':J_ms, 'K_Tms':K_Tms, 'tau_Pms':tau_Pms, 'V_A':V_A, 'w_Sms':w_Sms, 
                    'R_ms':R_ms, 'R_id':R_id, 'eta_Oms':eta_Oms, 'eta_Rms':eta_Rms, 't_ms':t_ms, 'w_Mms':w_Mms, 
                    'eta_Dms':eta_Dms, 'e_ims':e_ims, 'w_Sid':w_Sid}
    
    return trial_values, [K_T_coeffs, K_Q_coeffs]
    

Completing the trial phase values returns a dictionary of values. If vectors are entered instead of floats then a dictionary of arrays is returned.
Using this structure makes converting the output of the function in a pandas dataframe straightforword.

As the output is all scalers an idex must be passed to the dataframe commend. The index be the same length as the vectors returned by `calculate_all_values_from_trial_phase`.

In [ ]:
trial_values, K_coeffs = calculate_all_values_from_trial_phase(
            V_s = 8.8,
            P_dms = 14242e3,
            eta_ms = 1.018,
            delta_R = -44000,
            R_id = -44000,
            delta_eta  = 0,
            delta_t = 0,
            delta_w = 0,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            water_density = 1023,
            e_i = 1.089,
            t_Rid = 0.2,
            w_Mid = 0.24,
            J = np.linspace(0.3, 0.85, 12),
            K_T = np.asarray([0.3038, 0.2846, 0.2651, 0.2456, 0.226, 0.2064, 0.1867, 0.1669, 0.1467, 0.1264, 0.1056, 0.0845]),
            K_Q = np.asarray([0.397,0.3765,0.356,0.3352,0.3142,0.2929,0.2711,0.2488,0.2257,0.2016,0.1765,0.1501])/10
)

pd.DataFrame(trial_values, index = [0])

,K_Qms,J_ms,K_Tms,tau_Pms,V_A,w_Sms,R_ms,R_id,eta_Oms,eta_Rms,t_ms,w_Mms,eta_Dms,e_ims,w_Sid
0,0.018525,0.781956,0.11299,0.184789,7.444223,0.154066,1.182687e+06,1.226687e+06,0.759068,1.018,0.2,0.24,0.730771,1.113072,0.154066


In [ ]:
#| hide

trial_values, K_coeffs = calculate_all_values_from_trial_phase(
            V_s = 8.8,
            P_dms = 14242e3,
            eta_ms = 1.018,
            delta_R = -44000,
            R_id = -44000,
            delta_eta  = 0,
            delta_t = 0,
            delta_w = 0,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            water_density = 1023,
            e_i = 1.089,
            t_Rid = 0.2,
            w_Mid = 0.24,
            J = np.linspace(0.3, 0.85, 12),
            K_T = np.asarray([0.3038, 0.2846, 0.2651, 0.2456, 0.226, 0.2064, 0.1867, 0.1669, 0.1467, 0.1264, 0.1056, 0.0845]),
            K_Q = np.asarray([0.397,0.3765,0.356,0.3352,0.3142,0.2929,0.2711,0.2488,0.2257,0.2016,0.1765,0.1501])/10
)


test_close(trial_values['w_Sid'],0.154066, eps = 1e-5 )


### Calculate values in the ideal condition



In [ ]:
#| export
def calculate_all_values_from_ideal_phase(
        V_s:float,
        P_dms:float,
        delta_R:float,
        shaft_speed:float,
        diameter:float,
        number_shafts:float,
        t_Rid:float,
        R_id:float, 
        eta_Rms:float, 
        eta_Dms:float, 
        w_Sid:float,
        K_T_coeffs,
        K_Q_coeffs,
        water_density:float = 1026
        ): #returns a dictionaryt of the ouput variables from the trial condition calculations
    
    "intermediary function that calculates all the values in the ideal condition"

    tau_Pid = load_factor_resistance(R_id , t_Rid, w_Sid, V_s, diameter, water_density)/number_shafts

    #I don't really understand why the load factor uses the thrust coefficients
    J_id = propeller_advance_coefficient(tau_Pid , K_T_coeffs[0], K_T_coeffs[1], K_T_coeffs[2], "load"  )

    K_Tid = quadratic_method(K_T_coeffs, J_id)

    K_Qid = quadratic_method(K_Q_coeffs, J_id)

    eta_Oid = open_water_efficiency(J_id, K_Tid, K_Qid)

    n_id = propeller_speed(J_id, V_s, diameter, w_Sid)

    eta_Did = propulsive_efficiency_corr(eta_Oid , eta_Rms, t_Rid, w_Sid)

    delta_P = correction_delivered_power(P_dms, delta_R, V_s, eta_Did, eta_Dms)
    
    ideal_values = {'tau_Pid':tau_Pid, 'J_id':J_id, 'K_Tid':K_Tid, 'K_Qid':K_Qid, 'eta_Oid':eta_Oid, 'n_id':n_id , 'eta_Did':eta_Did, 'delta_P':delta_P}
    
    return ideal_values

In most cases the function will take the output of `calculate_all_values_from_trial_phase`. However there may be cases where some variables will be substituted. In the case below the total resistance in the ideal condition $R_{id}$ has been repplaced with another value with the rest of the values unchanged

In [ ]:

ideal_values = calculate_all_values_from_ideal_phase(
            V_s = 8.8,
            P_dms = 14242e3,
            delta_R = -44000,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            t_Rid = 0.2,
            R_id= 1.4e6, #trial_values['R_id'], #R_id is replaced with a different value 
            eta_Rms=trial_values['eta_Rms'],
            eta_Dms=trial_values['eta_Dms'],
            w_Sid=trial_values['w_Sid'],
            K_T_coeffs =  K_coeffs[0],
            K_Q_coeffs =  K_coeffs[1],
            water_density = 1026
)
ideal_values

{'tau_Pid': 0.21810388102065403,
 'J_id': 0.7546149357255287,
 'K_Tid': 0.12419788125885733,
 'K_Qid': 0.019860906443216052,
 'eta_Oid': 0.7510358183599756,
 'n_id': 1.1743963166125853,
 'eta_Did': 0.7230390006897096,
 'delta_P': -687825.7242918534}

However, it should be remembered that data can be entered as an array resulting in an array being returned. Also remember that the functions will perform broadcasting automatically.

In [ ]:
ideal_values = calculate_all_values_from_ideal_phase(
            V_s = 8.8,
            P_dms = 14242e3,
            delta_R = -44000,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            t_Rid = 0.2,
            R_id= np.array([1.8e6, 1.4e6, trial_values['R_id'] ]),
            eta_Rms=trial_values['eta_Rms'],
            eta_Dms=trial_values['eta_Dms'],
            w_Sid=trial_values['w_Sid'],
            K_T_coeffs =  K_coeffs[0],
            K_Q_coeffs =  K_coeffs[1],
            water_density = 1026
)

pd.DataFrame(ideal_values, index = [0,1,2])

,tau_Pid,J_id,K_Tid,K_Qid,eta_Oid,n_id,eta_Did,delta_P
0,0.280419,0.711273,0.141867,0.021944,0.731859,1.245959,0.704577,-1.079038e+06
1,0.218104,0.754615,0.124198,0.019861,0.751036,1.174396,0.723039,-6.878257e+05
2,0.191104,0.776506,0.115228,0.018793,0.757760,1.141288,0.729513,-5.553337e+05


In [ ]:
#| hide
ideal_values = calculate_all_values_from_ideal_phase(
            V_s = 8.8,
            P_dms = 14242e3,
            delta_R = -44000,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            t_Rid = 0.2,
            R_id= trial_values['R_id'], #R_id is replaced with a different value 
            eta_Rms=trial_values['eta_Rms'],
            eta_Dms=trial_values['eta_Dms'],
            w_Sid=trial_values['w_Sid'],
            K_T_coeffs =  K_coeffs[0],
            K_Q_coeffs =  K_coeffs[1],
            water_density = 1026
)

test_close(ideal_values['delta_P'], -555333, eps = 1)

### Power delivered in the ideal condition

This function brings all the rest of the functions in this module together into a single function call. Generally it would be easiest to use this. However, in the case that iteration is necessary, creating a custom function with a loop would be better. Such functionality may added to the library later as its use is better understood



In [ ]:

#| export
def delivered_power_ideal_condition(
    V_s:float,
    P_dms:float,
    eta_ms:float,
    delta_R:float,
    R_id:float,
    delta_eta :float,
    delta_t:float, 
    delta_w:float,
    shaft_speed:float,
    diameter:float,
    e_i:float,
    t_Rid:float,
    w_Mid:float,
    number_shafts:float,
    K_T,
    K_Q,
    J,
    water_density:float = 1026): #returns three results ideal_values, trial_values and the K coefficients
    
    "Calculate the delivered power in the Ideal conditions"

    trial_values, K_coeffs = calculate_all_values_from_trial_phase(
                V_s = V_s,
                P_dms =P_dms ,
                eta_ms = eta_ms ,
                delta_R = delta_R ,
                R_id = R_id,
                delta_eta = delta_eta,
                delta_t = delta_t,
                delta_w = delta_w ,
                shaft_speed = shaft_speed ,
                diameter = diameter,
                number_shafts = number_shafts ,
                water_density = water_density ,
                e_i = e_i,
                t_Rid = t_Rid,
                w_Mid = w_Mid ,
                J = J,
                K_T = K_T,
                K_Q = K_Q
    )






    ideal_values = calculate_all_values_from_ideal_phase(
                V_s = V_s,
                P_dms = P_dms,
                delta_R = delta_R ,
                shaft_speed = shaft_speed ,
                diameter = diameter, 
                number_shafts = number_shafts,
                t_Rid = t_Rid ,
                R_id = trial_values['R_id'], 
                eta_Rms = trial_values['eta_Rms'],
                eta_Dms = trial_values['eta_Dms'],
                w_Sid = trial_values['w_Sid'],
                K_T_coeffs =  K_coeffs[0],
                K_Q_coeffs =  K_coeffs[1],
                water_density = water_density
    )

    return ideal_values, trial_values, K_coeffs

Although this function produces a lot of output. Most can be ingored or used for checking results. 

In [ ]:
ideal_values, trial_values, K_coeffs = delivered_power_ideal_condition(
            V_s = 8.8,
            P_dms = 14242e3,
            eta_ms = 1.018,
            delta_R = -44000,
            R_id = -44000,
            delta_eta  = 0,
            delta_t = 0,
            delta_w = 0,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            water_density = 1023,
            e_i = 1.089,
            t_Rid = 0.2,
            w_Mid = 0.24,
            J = np.linspace(0.3, 0.85, 12),
            K_T = np.asarray([0.3038, 0.2846, 0.2651, 0.2456, 0.226, 0.2064, 0.1867, 0.1669, 0.1467, 0.1264, 0.1056, 0.0845]),
            K_Q = np.asarray([0.397,0.3765,0.356,0.3352,0.3142,0.2929,0.2711,0.2488,0.2257,0.2016,0.1765,0.1501])/10
)

ideal_values

{'tau_Pid': 0.191664144444155,
 'J_id': 0.7760288980241049,
 'K_Tid': 0.115424144090644,
 'K_Qid': 0.018816229604676736,
 'eta_Oid': 0.7576385853528017,
 'n_id': 1.1419896903753914,
 'eta_Did': 0.7293956323330646,
 'delta_P': -557713.3429971525}

In [ ]:
#| hide
ideal_values_all, trial_values_all, K_coeffs_all = delivered_power_ideal_condition(
            V_s = 8.8,
            P_dms = 14242e3,
            eta_ms = 1.018,
            delta_R = -44000,
            R_id = -44000,
            delta_eta  = 0,
            delta_t = 0,
            delta_w = 0,
            shaft_speed = 68/60, #rpm converted to sec
            diameter = 8.4, #because there are two shafts?
            number_shafts = 2,
            water_density = 1023,
            e_i = 1.089,
            t_Rid = 0.2,
            w_Mid = 0.24,
            J = np.linspace(0.3, 0.85, 12),
            K_T = np.asarray([0.3038, 0.2846, 0.2651, 0.2456, 0.226, 0.2064, 0.1867, 0.1669, 0.1467, 0.1264, 0.1056, 0.0845]),
            K_Q = np.asarray([0.397,0.3765,0.356,0.3352,0.3142,0.2929,0.2711,0.2488,0.2257,0.2016,0.1765,0.1501])/10
)

#
# Test fails why? there is a differnce of about 2000W power output
#
#test_close(ideal_values_all['delta_P']/1000, -555333/1000, eps = 1)

trial_values_all

{'K_Qms': 0.018525240056346072,
 'J_ms': 0.7819561773325622,
 'K_Tms': 0.11299045718360884,
 'tau_Pms': 0.1847893493004625,
 'V_A': 7.444222808205993,
 'w_Sms': 0.1540655899765918,
 'R_ms': 1182687.0763822165,
 'R_id': 1226687.0763822165,
 'eta_Oms': 0.7590676236822088,
 'eta_Rms': 1.018,
 't_ms': 0.2,
 'w_Mms': 0.24,
 'eta_Dms': 0.7307713995340193,
 'e_ims': 1.1130715921360634,
 'w_Sid': 0.1540655899765918}

## How it all hangs together

The relationships between the direct power equations are pretty complicated. The below flow diagram can help understand how it all hangs together

In [ ]:
'''{mermaid}
flowchart LR
  A[Hard edge] --> B(Round edge)
  B --> C{Decision}
  C --> D[Result one]
  C --> E[Result two]
'''

'{mermaid}\nflowchart LR\n  A[Hard edge] --> B(Round edge)\n  B --> C{Decision}\n  C --> D[Result one]\n  C --> E[Result two]\n'

In [ ]:
import nbdev; nbdev.nbdev_export()